# Let's get this party started

In [1]:
#Dependencies
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 1. Framing the Problem & Big Picture look (see readme file)

# 2. Get the Data

In [2]:
#set path to datasets
path = os.getcwd()
os.chdir("../datasets/")
filename = "sampledata.xlsx"
ser_path = "./serialized/"
filetwo = "sale_orders.xlsx"
filethree = "new_inventory.xlsx"

In [3]:
#Importing company datasets
#prod_hist = pd.read_excel(filename, sheet_name="production_hist")
#ship_hist = pd.read_excel(filename, sheet_name="shipment_hist")
#atp_hist = pd.read_excel(filename, sheet_name="atp_hist")
#inventory = pd.read_excel(filename, sheet_name="inventory")

In [4]:
#importing sales orders and new inventory
# order_hist = pd.read_excel(filetwo, sheet_name = "sale_orders")
# inv_hist2 = pd.read_excel(filethree, sheet_name = "new_inv")

In [5]:
# #setting file names
# data = [inventory, atp_hist, prod_hist, ship_hist]
# fnames = ["inventory", "atp_hist", "prod_hist","ship_hist"]
# count = 0
# for df in data:
#     df.name = fnames[count]
#     count+=1

In [6]:
# #saving dataframes into pickle files
# for df in data:
#     pklfile = df.name + ".pkl"
#     df.to_pickle(ser_path + pklfile)

In [7]:
# #setting file names
# data2 = [order_hist, inv_hist2]
# fnames = ["order_hist", "inv_hist2"]

# count = 0
# for df in data2:
#     df.name = fnames[count]
#     count+=1

# #saving new data into pickle files
# for df in data2:
#     pklfile = df.name + ".pkl"
#     df.to_pickle(ser_path + pklfile)

#saving dataframes into feather (not working)
for df in data:
    feafile = df.name
    df.to_feather(ser_path+feafile)

In [8]:
#see pklfile name order
pklfiles = os.listdir("./serialized")
pklfiles

['ship_hist.pkl',
 'inventory.pkl',
 'inv_hist2.pkl',
 'order_hist.pkl',
 'atp_hist.pkl',
 'prod_hist.pkl']

In [9]:
#Fast Upload using pickle files
ship_hist = pd.read_pickle(ser_path+pklfiles[0])
inventory = pd.read_pickle(ser_path+pklfiles[1])
inv_hist2 = pd.read_pickle(ser_path+pklfiles[2])
order_hist = pd.read_pickle(ser_path+pklfiles[3])
atp_hist = pd.read_pickle(ser_path+pklfiles[4])
prod_hist = pd.read_pickle(ser_path+pklfiles[5])

# 3. Cleaning Up

In [10]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["requested_date"] = dataframe["requested_date"].astype("datetime64")
        dataframe["confirmed_date"] = dataframe["confirmed_date"].astype("datetime64")
        dataframe["material_avail_date"] = dataframe["material_avail_date"].astype("datetime64")
        dataframe["load_date"] = dataframe["load_date"].astype("datetime64")
        dataframe["plan_goods_issue_date"] = dataframe["plan_goods_issue_date"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [11]:
# attempt to get rid of nulls, need to be more specific with fills to not loose valuable insight. 
# data = [inventory, atp_hist, prod_hist, ship_hist]
# for df in data:
#     df = df.fillna(0, inplace=True)

### Inventory

In [12]:
clean_columns(inventory)

Index(['calendar_day', 'plant', 'material', 'maktx', 'ph3', 'ph5', 'ph',
       'material_type', 'tons', 'shape', 'size_for_qual_cert', 'strgr',
       'block_resource', 'work_center', 'atp'],
      dtype='object')

In [13]:
inventory.head(5)

,calendar_day,plant,material,maktx,ph3,ph5,ph,material_type,tons,shape,size_for_qual_cert,strgr,block_resource,work_center,atp
0,2020-02-23,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
1,2020-02-20,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
2,2020-02-19,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
3,2020-02-18,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
4,2020-02-17,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN


In [14]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850873 entries, 0 to 850872
Data columns (total 15 columns):
calendar_day          850873 non-null datetime64[ns]
plant                 850873 non-null int64
material              850873 non-null int64
maktx                 850873 non-null object
ph3                   850873 non-null object
ph5                   850873 non-null object
ph                    850873 non-null object
material_type         850873 non-null object
tons                  850873 non-null float64
shape                 848401 non-null object
size_for_qual_cert    848401 non-null object
strgr                 850873 non-null int64
block_resource        626217 non-null object
work_center           850866 non-null float64
atp                   175364 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(8)
memory usage: 97.4+ MB


In [15]:
#Filling in empty atp rows with zero
inventory["atp"].fillna(0, inplace=True)

In [16]:
# inventory["work_center"] = inventory["work_center"].astype("int64")

In [17]:
inventory = inventory.loc[inventory["material_type"]=="ZERT"]
inventory["material_type"].unique()

array(['ZERT'], dtype=object)

In [18]:
inv_reduced = inventory[["calendar_day", "strgr", "material", "maktx", "work_center", "ph", "shape", "size_for_qual_cert", "block_resource", "tons", "atp"]]
inv_reduced.rename(columns={"material":"sku", "maktx":"material_description", "size_for_qual_cert":"size", "block_resource":"block", "strgr":"strategy"}, inplace=True)
inv_reduced = inv_reduced.sort_values(["calendar_day", "material_description"])
inv_reduced.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,calendar_day,strategy,sku,material_description,work_center,ph,shape,size,block,tons,atp
799352,2017-01-01,63,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,9.442000,0.0
799094,2017-01-01,63,102003053,"BL 5 1/8X7 3/8 1010M-446_1302 3,858 SA",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,1.929000,0.0
799267,2017-01-01,63,102003353,BL 5 1/8X7 3/8 GAU_1050-501_1302 3168 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,7.920000,0.0
799180,2017-01-01,63,102003354,BL 5 1/8X7 3/8 GAU_MG12-500_1302 3220 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,11.380000,0.0
844399,2017-01-01,40,106020238,"C 10X15.3# GGMULTI 20'00""",25000001.0,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,10 X 15.3#,"10"" CHN",110.007004,0.0


In [19]:
#changing dtype
inv_reduced["strategy"] = inv_reduced["strategy"].astype("category")
inv_reduced["sku"] = inv_reduced["sku"].astype("object")
inv_reduced["work_center"] = inv_reduced["work_center"].astype("category")

In [20]:
#view of new dtypes & counts
inv_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649986 entries, 799352 to 18367
Data columns (total 11 columns):
calendar_day            649986 non-null datetime64[ns]
strategy                649986 non-null category
sku                     649986 non-null object
material_description    649986 non-null object
work_center             649979 non-null category
ph                      649986 non-null object
shape                   647514 non-null object
size                    647514 non-null object
block                   626217 non-null object
tons                    649986 non-null float64
atp                     649986 non-null float64
dtypes: category(2), datetime64[ns](1), float64(2), object(6)
memory usage: 50.8+ MB


In [21]:
#peak at all nulls across
inv_reduced[inv_reduced.isna().any(axis=1)]

,calendar_day,strategy,sku,material_description,work_center,ph,shape,size,block,tons,atp
799352,2017-01-01,63,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,9.442,0.0
799094,2017-01-01,63,102003053,"BL 5 1/8X7 3/8 1010M-446_1302 3,858 SA",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,1.929,0.0
799267,2017-01-01,63,102003353,BL 5 1/8X7 3/8 GAU_1050-501_1302 3168 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,7.920,0.0
799180,2017-01-01,63,102003354,BL 5 1/8X7 3/8 GAU_MG12-500_1302 3220 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,11.380,0.0
840484,2017-01-01,40,106020149,"C 12X30# GGMULTI 20'00""",25000001.0,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,12 X 30#,NaN,1.200,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3819,2020-02-23,40,109004211,"PR 3/8"" 1018 20'00"" CD",25000001.0,LS-Mills-Mer-Mer-MBQ-Pencil Rod,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
19798,2020-02-23,63,205012261,"RD .678"" 1055MC 31'00""",25000002.0,SS-Mills-SBQ-SBQ-SBQ-SBQ Rd,ROUND,".678""",NaN,-0.001,0.0
8136,2020-02-23,40,109250351,"RD 1 1/4"" GGMULTI 20'00""",25000002.0,LS-Mills-Mer-Mer-MBQ-MBQ Rd,ROUND,"1 1/4""",NaN,2.529,0.0
8134,2020-02-23,63,109250353,"RD 1/2"" GGMULTI 20'00""",25000002.0,LS-Mills-Mer-Mer-MBQ-MBQ Rd,ROUND,"1/2""",NaN,-0.002,0.0


### ATP History

In [22]:
clean_columns(atp_hist)

Index(['plant', 'plant_description', 'sku', 'atp_floor', 'snapshot_date'], dtype='object')

In [23]:
atp_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235706 entries, 0 to 235705
Data columns (total 5 columns):
plant                235706 non-null int64
plant_description    235706 non-null object
sku                  235706 non-null object
atp_floor            235706 non-null float64
snapshot_date        235706 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 9.0+ MB


In [24]:
atp_reduced = atp_hist[["sku", "atp_floor", "snapshot_date"]]
atp_reduced.head()

,sku,atp_floor,snapshot_date
0,102003353,2.0,2019-11-01
1,102003353,2.0,2019-10-17
2,102003353,2.0,2019-10-16
3,102003353,2.0,2019-10-15
4,102003353,2.0,2019-10-14


In [25]:
atp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235706 entries, 0 to 235705
Data columns (total 3 columns):
sku              235706 non-null object
atp_floor        235706 non-null float64
snapshot_date    235706 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 5.4+ MB


### Production History

In [26]:
clean_columns(prod_hist)

Index(['calendar_day', 'plant', 'plant_description', 'production_process',
       'production_process_description', 'work_center', 'wc_description',
       'material', 'material_description', 'ph3', 'ph5', 'ton', 'block',
       'shape', 'size', 'stratedgy', 'month/year', 'year', 'monthyear'],
      dtype='object')

In [27]:
prod_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41935 entries, 0 to 41934
Data columns (total 19 columns):
calendar_day                      41935 non-null datetime64[ns]
plant                             41935 non-null int64
plant_description                 41935 non-null object
production_process                41935 non-null int64
production_process_description    41935 non-null object
work_center                       37833 non-null object
wc_description                    41935 non-null object
material                          41935 non-null int64
material_description              41935 non-null object
ph3                               36579 non-null object
ph5                               36579 non-null object
ton                               41935 non-null int64
block                             14851 non-null object
shape                             36115 non-null object
size                              36115 non-null object
stratedgy                         36129 non-nul

In [28]:
# prod_hist["stratedgy"] = prod_hist["stratedgy"].astype("int64")

In [29]:
prod_reduced = prod_hist[["calendar_day", "production_process", "production_process_description","stratedgy","work_center","wc_description","block","shape","size","material","material_description","ton"]]
prod_reduced.rename(columns={"material":"sku", "stratedgy":"strategy"}, inplace=True)
prod_reduced.head()

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
0,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106023315,"L 4X4X3/8 44W 60'00""",40
1,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022771,"L 4X4X3/8 A36/44W/A529-50 40'00""",390
2,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022760,"L 4X4X3/8 A36/44W/A529-50 20'00""",345
3,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022755,"L 4X4X3/8 55W 60'00""",205
4,2013-02-06,4310,Rolling,63.0,25000001,ROLLING - STRUCTURAL,"4"" ANG",SHAPE_L,4X4X3/8,106022749,"L 4X4X3/8 50W 60'00""",32


In [30]:
#Removing strategy 10 (Note: check assumption that these are all billets)
prod_reduced = prod_reduced.loc[(prod_reduced["strategy"] !=10)]

In [31]:
#Removing prod_processes up the supply chain: melt, scrap etc.
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Melt shop") & (prod_reduced["production_process_description"] != "Scrap yard")]
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Finishing production")]

In [32]:
#Identifying strategy=NaN rows
prod_reduced[prod_reduced["strategy"].isna()]

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
36154,2015-02-27,4310,Rolling,NaN,25000002,ROLLING - BAR,NaN,NaN,NaN,40000148,COPROD - FG YIELD LOSS (ROLLING MILL),10
36155,2016-12-29,4310,Rolling,NaN,25000002,ROLLING - BAR,NaN,NaN,NaN,40000148,COPROD - FG YIELD LOSS (ROLLING MILL),-10


In [33]:
#Identifying size=NaN rows
#prod_reduced[prod_reduced["size"].isna()]

In [34]:
#Dropping selective NaN rows
prod_reduced.dropna(subset=["strategy"], inplace=True)

In [35]:
#Identifying unique inputs by column
unique_counts = pd.DataFrame.from_records([(col, prod_reduced[col].nunique()) for col in prod_reduced.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

,Column_Name,Num_Unique
1,production_process,1
2,production_process_description,1
3,strategy,2
4,work_center,4
5,wc_description,4
7,shape,6
6,block,41
8,size,175
11,ton,1299
9,sku,1862


In [36]:
# Changing data types for colums, 
for col in prod_reduced.columns:
    if prod_reduced[col].nunique() < 10:
        prod_reduced[col] = prod_reduced[col].astype("category")
# not including "block" as it used as object elsewhere

prod_reduced["sku"] =prod_reduced["sku"].astype("object")

In [37]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14820 entries, 0 to 36135
Data columns (total 12 columns):
calendar_day                      14820 non-null datetime64[ns]
production_process                14820 non-null category
production_process_description    14820 non-null category
strategy                          14820 non-null category
work_center                       13450 non-null category
wc_description                    14820 non-null category
block                             14815 non-null object
shape                             14795 non-null category
size                              14795 non-null object
sku                               14820 non-null object
material_description              14820 non-null object
ton                               14820 non-null int64
dtypes: category(6), datetime64[ns](1), int64(1), object(4)
memory usage: 898.1+ KB


In [38]:
prod_reduced[prod_reduced["block"].isna()]["production_process_description"]

35617    Rolling
35618    Rolling
35631    Rolling
35632    Rolling
36135    Rolling
Name: production_process_description, dtype: category
Categories (1, object): [Rolling]

In [39]:
prod_reduced[prod_reduced["block"].isna()].groupby("production_process_description").count()

,calendar_day,production_process,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
production_process_description,,,,,,,,,,,
Rolling,5,5,5,3,5,0,5,5,5,5,5


### Ship History

In [40]:
clean_columns(ship_hist)

Index(['shipping_day', 'plant', 'plant_description', 'sales_organization',
       'sales_doc_type', 'outside_rep_ship-to', 'inside_rep_ship-to',
       'sold-to_party', 'sold-to_part_name', 'ship-to_party',
       'ship-to_party_name', 'postal_code', 'location', 'region',
       'region_description', 'country', 'country_description', 'material',
       'material_description', 'div_id', 'div', 'prod_hier_level_3',
       'prod_hier_level_5', 'shipment_tons', 'manager', 'segment',
       'plantcountry', 'businessdaycounter', 'block', 'lvorm', 'mrptype',
       'proctype', 'relativemonth', 'shipping_week', 'strgr', 'shape',
       'sizeforqualcert', 'dailyshiprate3', 'monthdelta', 'dailyshiprate6',
       'dailyshiprate12', 'workcenter', 'wc_desc', 'bayou_flag'],
      dtype='object')

In [41]:
ship_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156796 entries, 0 to 156795
Data columns (total 44 columns):
shipping_day            156796 non-null datetime64[ns]
plant                   156796 non-null int64
plant_description       156796 non-null object
sales_organization      156796 non-null object
sales_doc_type          156796 non-null object
outside_rep_ship-to     156796 non-null object
inside_rep_ship-to      156796 non-null object
sold-to_party           156796 non-null int64
sold-to_part_name       156796 non-null object
ship-to_party           156796 non-null int64
ship-to_party_name      156796 non-null object
postal_code             156796 non-null object
location                156796 non-null object
region                  156796 non-null object
region_description      156796 non-null object
country                 156796 non-null object
country_description     156796 non-null object
material                156796 non-null int64
material_description    156796 non-null

In [42]:
#Filling in NaN Shipment field rows to zero
ship_hist["shipment_tons"].fillna(0, inplace=True)

In [43]:
# Attempt to get correct data types, needs work.
# ship_hist["businessdaycounter"] = ship_hist["businessdaycounter"].astype("int64")
# ship_hist["strgr"] = ship_hist["strgr"].astype("int64")
# ship_hist["workcenter"] = ship_hist["workcenter"].astype("int64")

In [44]:
#Reducing main data frame
ship_reduced = ship_hist[["shipping_day", "block", "strgr", "shape", "sizeforqualcert", "div", "material", "material_description","shipment_tons", "segment", "sold-to_party","ship-to_party", "postal_code","region_description", "country"]]
ship_reduced.rename(columns={"material":"sku", "strgr":"strategy","sizeforqualcert":"size"}, inplace=True)
ship_reduced.head()

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
0,2020-01-29,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
1,2020-01-07,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
2,2020-01-24,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021716,"C 8X11.5# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092318,100099655,37218-3813,Tennessee,US
3,2020-01-29,"3 X 2"" ANG",40.0,SHAPE_L,3X2X1/2,Merchants,106022002,"L 3X2X1/2 GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092779,100099953,72301-7103,Arkansas,US
4,2020-01-27,"12"" CHN",40.0,SHAPE_C,12 X 20.7#,Merchants,106020260,"C 12X20.7# GGMULTI 40'00""",0.0,NA - DISTRIBUTION,100092309,100095766,35954,Alabama,US


In [45]:
#Check unique Divisions
ship_hist["div"].unique()

array(['Merchants', 'Rebar', 'Billets', 'SBQ Bar', 'Scrap'], dtype=object)

In [46]:
#Removing billets and scrap from dataframe
ship_reduced = ship_reduced.loc[(ship_reduced["div"] != "Billets") & (ship_reduced["div"] != "Scrap")]

In [47]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156711 entries, 0 to 156795
Data columns (total 15 columns):
shipping_day            156711 non-null datetime64[ns]
block                   153079 non-null object
strategy                156700 non-null float64
shape                   156700 non-null object
size                    156700 non-null object
div                     156711 non-null object
sku                     156711 non-null int64
material_description    156711 non-null object
shipment_tons           156711 non-null float64
segment                 154025 non-null object
sold-to_party           156711 non-null int64
ship-to_party           156711 non-null int64
postal_code             156711 non-null object
region_description      156711 non-null object
country                 156711 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(9)
memory usage: 19.1+ MB


In [48]:
#Show Sample of NaN fields
ship_reduced[ship_reduced.isna().any(axis=1)]

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
91,2020-01-28,"2-1/2"" ANG",40.0,SHAPE_L,2 1/2X2 1/2X1/4,Merchants,106028782,"L 2 1/2X2 1/2X1/4 GGMULTI 20'00""",0.0,NaN,100051158,100051158,77029,Texas,US
152,2019-10-29,"2-1/2"" ANG",40.0,SHAPE_L,2 1/2X2 1/2X1/4,Merchants,106028782,"L 2 1/2X2 1/2X1/4 GGMULTI 20'00""",0.0,NaN,100051158,100051158,77029,Texas,US
314,2020-01-06,NaN,40.0,FLAT,1/4 X 5,Merchants,109005365,"F 1/4X5 GGMULTI 20'00""",0.0,NA - DISTRIBUTION,100091890,100094606,N2R 1E6,Ontario,CA
430,2020-02-04,"8"" FLT",40.0,FLAT,1/2 X 8,Merchants,109008964,"F 1/2X8 GGMULTI 20'00""",0.0,NaN,100092155,100478827,S6V 5P9,Saskatchewan,CA
431,2020-02-04,"2"" ANG",40.0,SHAPE_L,2X2X1/8,Merchants,106029077,"L 2X2X1/8 GGMULTI 20'00""",0.0,NaN,100092155,100478827,S6V 5P9,Saskatchewan,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155890,2017-05-24,NaN,40.0,SHAPE_C,15 X 50#,Merchants,106027672,"C 15X50# GGMULTI 40'00""",24.0,NA - DISTRIBUTION,100091238,100103765,14606-3100,New York,US
156012,2017-02-08,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106019519,"C 15X33.9# GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100060362,100060377,49444-2333,Michigan,US
156429,2017-03-22,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106027715,"C 15X33.9# GGMULTI 50'00""",8.0,NA - DISTRIBUTION,100060244,100060250,40209-1845,Kentucky,US
156455,2017-01-31,NaN,40.0,SHAPE_C,15 X 33.9#,Merchants,106019519,"C 15X33.9# GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100060244,100114121,60440,Illinois,US


In [49]:
#Show the shape that show as null within the block
ship_reduced[ship_reduced["block"].isna()].groupby("shape").count()

,shipping_day,block,strategy,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
shape,,,,,,,,,,,,,,
ASDRAWING,2,0,2,2,2,2,2,2,2,2,2,2,2,2
FLAT,2457,0,2457,2457,2457,2457,2457,2457,2249,2457,2457,2457,2457,2457
PENCIL_ROD,81,0,81,81,81,81,81,81,81,81,81,81,81,81
ROUND,246,0,246,246,246,246,246,246,216,246,246,246,246,246
SHAPE_C,237,0,237,237,237,237,237,237,231,237,237,237,237,237
SHAPE_L,415,0,415,415,415,415,415,415,384,415,415,415,415,415
SHAPE_MC,31,0,31,31,31,31,31,31,31,31,31,31,31,31
SQUARE,152,0,152,152,152,152,152,152,139,152,152,152,152,152


### Sales Order Data

In [50]:
clean_columns(order_hist)

Index(['sold_to', 'ship_to', 'ship_to_city', 'ship_to_state', 'order_no',
       'item_no', 'material_number', 'material_description', 'division',
       'reject_reason_code', 'po_line_item_no', 'cust_material_no',
       'overdeliv_tol', 'underdeliv_tol', 'plant', 'plant_name', 'storage_loc',
       'shipping_point', 'route', 'sales_district_code', 'requested_date',
       'confirmed_date', 'order_qty_sales_units', 'confirmed_qty', 'sales_uom',
       'weight_per_piece', 'req_qty_base_uom', 'base_uom', 'transp_plan_date',
       'material_avail_date', 'load_date', 'plan_goods_issue_date',
       'delivery_block', 'qty_on_delivery', 'qty_on_shipment', 'qty_shipped',
       'qty_on_delivery_sales_units', 'qty_on_shipment_sales_units',
       'qty_shipped_sales_units', 'item_status'],
      dtype='object')

In [51]:
#Reducing main data frame
order_reduced = order_hist[["requested_date",'confirmed_date','material_avail_date', 'load_date', 'plan_goods_issue_date','sales_district_code',
                            "division", "sold_to", "ship_to","route","ship_to_city", "ship_to_state", "material_number",'material_description',
                            'order_qty_sales_units', 'confirmed_qty', 'sales_uom','qty_on_shipment','qty_shipped_sales_units', 'item_status',]]
order_reduced.rename(columns={"material_number":"sku"}, inplace=True)
order_reduced.head(-5)

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,ship_to_city,ship_to_state,sku,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status
0,2012-10-25,2013-02-08,2013-02-06,2013-02-06,2013-02-07,,5,100091881,100103709,C00731,CANDIAC,QC,106020877,"L 6X6X1/2 A36/44W/A529-50 40'00""",372.000,372.000,ST,0.0,82176.000,Completely Processed
1,2012-10-25,2013-04-01,2013-03-27,2013-03-27,2013-03-28,,5,100091881,100103709,C00731,CANDIAC,QC,106020931,"L 6X6X3/8 50W 46'05""",240.000,240.000,ST,0.0,116504.016,Completely Processed
2,2012-10-25,2012-12-18,2012-12-14,2012-12-14,2012-12-17,,5,100091881,100103709,C00731,CANDIAC,QC,106020931,"L 6X6X3/8 50W 46'05""",66.003,66.003,ST,0.0,91296.000,Completely Processed
3,2013-04-16,2013-07-03,2013-06-03,2013-06-03,2013-06-18,,3,100092158,100099545,CM0163,NISKU,AB,106021399,"C 6X8.2# A36/44W/A529-50 40'00""",90.000,90.000,ST,0.0,180.000,Completely Processed
4,2013-04-16,2013-07-10,2013-06-07,2013-06-07,2013-06-24,,3,100092158,100099545,CM0163,NISKU,AB,106021716,"C 8X11.5# GGMULTI 40'00""",20.000,20.000,ST,0.0,40.000,Completely Processed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188943,2020-05-29,2020-05-29,2020-04-29,2020-04-29,2020-05-14,100209,2,100092222,100095545,CM0303,SASKATOON,SK,110007718,"MR 20M 400W 48'06"" 9,100MA",180243.000,180243.000,LB,NaN,NaN,Not Yet Processed
188944,2020-06-05,2020-06-05,2020-05-06,2020-05-06,2020-05-21,100209,2,100092222,100095545,CM0303,SASKATOON,SK,110007718,"MR 20M 400W 48'06"" 9,100MA",180243.000,180243.000,LB,NaN,NaN,Not Yet Processed
188945,2020-08-17,2020-08-17,2020-07-15,2020-07-15,2020-07-30,100312,3,100092142,100095283,CM0587,ABBOTSFORD,BC,106028208,"C 7X9.8# GGMULTI 40'00""",9408.000,9408.000,LB,NaN,NaN,Not Yet Processed
188946,2020-06-13,2020-06-13,2020-05-08,2020-05-08,2020-05-26,100312,3,100092142,100095283,CM0587,ABBOTSFORD,BC,109008152,"F 3/8X6 GGMULTI 20'00""",5972.000,5972.000,LB,NaN,NaN,Not Yet Processed


### New Inventory Data (dating back 2016)

In [63]:
inv_hist2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779395 entries, 0 to 779394
Data columns (total 23 columns):
calendar_day          779395 non-null datetime64[ns]
plant                 779395 non-null int64
material              779395 non-null int64
ph3                   779395 non-null object
ph5                   779395 non-null object
ph                    779395 non-null object
material_type         779395 non-null object
tons                  779395 non-null float64
matnr                 779395 non-null int64
maktx                 779395 non-null object
shape                 776588 non-null object
size_for_qual_cert    776588 non-null object
mto_mts               771934 non-null object
planning_group        776585 non-null object
ptp_group             776588 non-null object
mill_combo_code       779392 non-null object
pg_group_code         779392 non-null object
block_resource        769251 non-null object
l5code                779392 non-null object
steel_grade           776588

In [56]:
clean_columns(inv_hist2) 

Index(['calendar_day', 'plant', 'material', 'ph3', 'ph5', 'ph',
       'material_type', 'tons', 'matnr', 'maktx', 'shape',
       'size_for_qual_cert', 'mto_mts', 'planning_group', 'ptp_group',
       'mill_combo_code', 'pg_group_code', 'block_resource', 'l5code',
       'steel_grade', 'length_imperial', 'length_text', 'weight_per_piece'],
      dtype='object')

In [75]:
inv_reduced2 = inv_hist2[["calendar_day","material", "maktx","material_type", "mto_mts", "ph","shape", "block_resource",
                          "size_for_qual_cert", "steel_grade","length_text", "tons"]]
inv_reduced2.rename(columns={"material":"sku", "maktx":"material_description", "mto_mts":"strategy", "block_resource":"block", 
                             "size_for_qual_cert":"size"}, inplace=True)
inv_reduced2 = inv_reduced2.sort_values(["calendar_day", "material_description"])
inv_reduced2.head()

,calendar_day,sku,material_description,material_type,strategy,ph,shape,block,size,steel_grade,length_text,tons
0,2016-01-03,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",ZERT,MTO,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,NaN,5 1/8 X 7 3/8,1010M-446_1302,0,7.600
16,2016-01-03,106020238,"C 10X15.3# GGMULTI 20'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"20'00""",200.736
17,2016-01-03,106020239,"C 10X15.3# GGMULTI 40'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"40'00""",352.206
18,2016-01-03,106020242,"C 10X15.3# GGMULTI 50'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"50'00""",99.450
19,2016-01-03,106020243,"C 10X15.3# GGMULTI 60'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"60'00""",26.622


In [78]:
#changing dtype
inv_reduced2["strategy"] = inv_reduced2["strategy"].astype("category")
inv_reduced2["sku"] = inv_reduced2["sku"].astype("object")

In [86]:
#removing non-end use product hierarchy
inv_reduced2 = inv_reduced2[(inv_reduced2["ph"]!= "LS-Mills-Bil-Bil-Bil-Billets") & 
                            (inv_reduced2["ph"]!= "DO NOT USE") &
                            (inv_reduced2["ph"]!= "LS-Mills-Mer-Mer-MBQ-MBQ HRB")]
inv_reduced2["ph"].value_counts()

LS-Mills-Mer-Mer-Strc A&C-Angles          263153
LS-Mills-Mer-Mer-Lght Shp-MBQ Angles      174981
LS-Mills-Mer-Mer-Strc A&C-Channels        164836
LS-Mills-Mer-Mer-MBQ-MBQ Fl                70798
LS-Mills-Reb-Reb-Str Reb-Str Reb           41162
LS-Mills-Mer-Mer-Lght Shp-MBQ Channels     37024
LS-Mills-Reb-Reb-Reb Spc-Rebar MRB          9864
LS-Mills-Mer-Mer-Strc A&C-MC Channels       6594
LS-Mills-Mer-Mer-MBQ-MBQ Rd                 3091
LS-Mills-Mer-Mer-MBQ-MBQ Sq                 1748
LS-Mills-Mer-Mer-MBQ-Pencil Rod              895
AL-Laminaci√≥n-Perfiles-Canal                277
SS-Mills-SBQ-SBQ-SBQ-SBQ Rd                  170
Name: ph, dtype: int64

In [87]:
inv_reduced2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774593 entries, 16 to 779020
Data columns (total 12 columns):
calendar_day            774593 non-null datetime64[ns]
sku                     774593 non-null object
material_description    774593 non-null object
material_type           774593 non-null object
strategy                767339 non-null category
ph                      774593 non-null object
shape                   771786 non-null object
block                   769246 non-null object
size                    771786 non-null object
steel_grade             771786 non-null object
length_text             769268 non-null object
tons                    774593 non-null float64
dtypes: category(1), datetime64[ns](1), float64(1), object(9)
memory usage: 71.7+ MB


### Cleaned Reduced Datasets

In [88]:
data_reduced = [ship_reduced, inv_reduced, atp_reduced, prod_reduced, order_reduced, inv_reduced2]

In [89]:
#storing cleaned Prod
%store prod_reduced

#storing cleaned Ship
%store ship_reduced

#storing cleaned Inventory
#%store inv_reduced

#storing cleaned ATP
%store atp_reduced

#storing orders
%store order_reduced

#storing new(2016) inventory
%store inv_reduced2

Stored 'prod_reduced' (DataFrame)
Stored 'ship_reduced' (DataFrame)
Stored 'atp_reduced' (DataFrame)
Stored 'order_reduced' (DataFrame)
Stored 'inv_reduced2' (DataFrame)


# 4. Merging to single Dataframe 